<a href="https://colab.research.google.com/github/Elabs-llc/Elabs-llc/blob/main/VPN_Client_for_Windows_(Python).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# vpn_client.py
# A simplified proof-of-concept VPN client with a basic GUI.
# WARNING: This is for educational purposes ONLY and is NOT secure.

import socket
import threading
import os
import tkinter as tk
from tkinter import scrolledtext, messagebox
from cryptography.fernet import Fernet

# --- Configuration ---
# IMPORTANT: Change this to the public IP address of your server
SERVER_HOST = '127.0.0.1'
SERVER_PORT = 8888
KEY = b''

# --- Global Variables ---
client_socket = None
is_connected = False
fernet = None

# --- Load Encryption Key ---
def load_key():
    """
    Loads the encryption key from 'secret.key'.
    This file must be copied from the server to the client.
    """
    global KEY, fernet
    if not os.path.exists('secret.key'):
        messagebox.showerror("Error", "Encryption key file 'secret.key' not found.\nPlease copy it from the server.")
        return False

    with open('secret.key', 'rb') as key_file:
        KEY = key_file.read()
    fernet = Fernet(KEY)
    log("Encryption key loaded.")
    return True

# --- Core VPN Functions ---
def connect_to_server():
    """
    Connects to the VPN server and updates the UI.
    """
    global client_socket, is_connected
    if is_connected:
        log("Already connected.")
        return

    log(f"Attempting to connect to {SERVER_HOST}:{SERVER_PORT}...")
    try:
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        client_socket.connect((SERVER_HOST, SERVER_PORT))
        is_connected = True
        log("Successfully connected to the VPN server!")
        update_ui_status()
    except Exception as e:
        log(f"Connection failed: {e}")
        messagebox.showerror("Connection Failed", f"Could not connect to the server.\nError: {e}")
        is_connected = False
        client_socket = None
        update_ui_status()

def disconnect_from_server():
    """
    Disconnects from the VPN server and updates the UI.
    """
    global client_socket, is_connected
    if not is_connected:
        log("Already disconnected.")
        return

    try:
        client_socket.close()
    except Exception as e:
        log(f"Error while disconnecting: {e}")
    finally:
        is_connected = False
        client_socket = None
        log("Disconnected from the VPN server.")
        update_ui_status()

def send_request():
    """
    Simulates sending a request through the VPN.
    In this example, it sends a simple HTTP GET request for google.com.
    """
    if not is_connected or not fernet:
        messagebox.showwarning("Not Connected", "Please connect to the VPN server first.")
        return

    # A simple HTTP GET request. A real VPN would capture this from the browser.
    http_request = b"GET / HTTP/1.1\r\nHost: www.google.com\r\nConnection: close\r\n\r\n"

    log("Encrypting and sending a sample request to google.com...")
    try:
        # Encrypt the request
        encrypted_request = fernet.encrypt(http_request)

        # Send it to our VPN server
        client_socket.sendall(encrypted_request)

        # Wait for the encrypted response from our VPN server
        encrypted_response = client_socket.recv(8192) # Increased buffer

        # Decrypt the response
        decrypted_response = fernet.decrypt(encrypted_response)

        log("Received and decrypted response:")
        # Display the first 500 characters of the response
        log(decrypted_response.decode('utf-8', 'ignore')[:500] + "...")

    except Exception as e:
        log(f"An error occurred during the request: {e}")
        disconnect_from_server()


# --- GUI Functions ---
def log(message):
    """Adds a message to the log window in a thread-safe way."""
    log_area.config(state=tk.NORMAL)
    log_area.insert(tk.END, message + "\n")
    log_area.config(state=tk.DISABLED)
    log_area.see(tk.END)

def update_ui_status():
    """Updates button states and status label based on connection status."""
    if is_connected:
        status_label.config(text="Status: Connected", fg="green")
        connect_button.config(state=tk.DISABLED)
        disconnect_button.config(state=tk.NORMAL)
        request_button.config(state=tk.NORMAL)
    else:
        status_label.config(text="Status: Disconnected", fg="red")
        connect_button.config(state=tk.NORMAL)
        disconnect_button.config(state=tk.DISABLED)
        request_button.config(state=tk.DISABLED)

def on_closing():
    """Handles window close event."""
    if is_connected:
        disconnect_from_server()
    root.destroy()

# --- GUI Setup ---
root = tk.Tk()
root.title("Simple PoC VPN Client")
root.geometry("600x400")

# Main Frame
main_frame = tk.Frame(root, padx=10, pady=10)
main_frame.pack(fill=tk.BOTH, expand=True)

# Status Label
status_label = tk.Label(main_frame, text="Status: Disconnected", fg="red", font=("Helvetica", 12, "bold"))
status_label.pack(pady=5)

# Buttons Frame
button_frame = tk.Frame(main_frame)
button_frame.pack(pady=10)

connect_button = tk.Button(button_frame, text="Connect", command=lambda: threading.Thread(target=connect_to_server).start())
connect_button.pack(side=tk.LEFT, padx=5)

disconnect_button = tk.Button(button_frame, text="Disconnect", command=disconnect_from_server, state=tk.DISABLED)
disconnect_button.pack(side=tk.LEFT, padx=5)

request_button = tk.Button(main_frame, text="Send Sample Request (to Google)", command=lambda: threading.Thread(target=send_request).start(), state=tk.DISABLED)
request_button.pack(pady=10)

# Log Area
log_area = scrolledtext.ScrolledText(main_frame, state=tk.DISABLED, wrap=tk.WORD, height=15)
log_area.pack(fill=tk.BOTH, expand=True, pady=5)

# --- Start Application ---
if __name__ == "__main__":
    if not load_key():
        root.destroy()
    else:
        root.protocol("WM_DELETE_WINDOW", on_closing)
        root.mainloop()